In [1]:
import pandas as pd
import numpy as np
import os
import dask
import dask.dataframe as dd
import itertools
from itertools import chain
from math import sqrt, floor, ceil, isnan
import multiprocess
import importlib
from importlib import reload
from collections import Counter
from fuzzywuzzy import process, fuzz
import time
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("error")

pd.options.display.max_columns = 500
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = 400

try:
    del(FUN_proc_name)
except:
    pass
import FUN_proc_name
importlib.reload(FUN_proc_name)
from FUN_proc_name import FUN_proc_name

# Whether to run the step of merging GPF underwriters to SDC M&A deals. Note that M&As from SDC has been hand-cleaned and 
# recorded in "SCRIPT_SDC_deals_cleaned.csv"
IF_match_SDC_MA = False
IF_fuzzy_match = False

In [2]:
# Import GPF
GPF = pd.read_csv("../RawData/SDC/GPF.csv",low_memory=False)
GPF = GPF.drop(columns=['Unnamed: 0'])

# Generate cleaned names
raw_name_GPF_colnames = [column for column in GPF.columns if 'raw_name_GPF_' in column]
idx = 0
for column in raw_name_GPF_colnames:
    GPF['name_GPF_'+str(idx)] = GPF[column].apply(FUN_proc_name)
    idx = idx+1
name_GPF_colnames = ['name_GPF_'+str(idx) for idx in range(0,len(raw_name_GPF_colnames))]

# Hand corrections
# (1) Change "CHEMICAL BANK" to "CHEMICAL BANK MICHIGAN" for deals in MI
for column in raw_name_GPF_colnames:
    GPF.loc[(GPF['raw_name_GPF_0'].str.contains('Chemical Bank'))&(GPF['State']=='MI'),column] = 'CHEMICAL BANK MICHIGAN'

In [3]:
GPF_names = pd.read_parquet('../CleanData/SDC/GPF_names.parquet')
MA = pd.read_parquet('../CleanData/SDC/M&A.parquet')

In [4]:
def export_fate(state,begin_year,end_year,begin_rank,end_rank):

    name_GPFs = list(chain.from_iterable(list(np.array(
        GPF[(GPF['sale_year']>=begin_year)&(GPF['sale_year']<=end_year)&(GPF['State']==state)][name_GPF_colnames]))))
    name_GPFs = [item for item in name_GPFs if item!=None]
    name_GPFs = [item for item in name_GPFs if str(item)!='nan']
    name_GPFs = Counter(name_GPFs).most_common()
    
    table_data = []
    for item in name_GPFs[begin_rank:end_rank]:
        MA_frag = MA[MA['target']==item[0]].reset_index()
        # Whether it is ever aquired
        if len(MA_frag)==0:
            table_data = table_data+[[str(item[1]),item[0].replace('&',' ')]]
        elif len(MA_frag)==1:
            acquiror = MA_frag['acquiror'][0]
            MA_frag = MA[MA['target']==acquiror].reset_index()
            # Check if parent is acquired again
            if len(MA_frag)==0:
                one_bank_data = [str(item[1]),item[0].replace('&',' ')+' $\\rightarrow$',acquiror.replace('&',' ')]
                table_data = table_data+[one_bank_data]
            else:
                second_acquiror = MA_frag['acquiror'][0]
                MA_frag = MA[MA['target']==second_acquiror].reset_index()
                # Check if parent is acquired again
                if len(MA_frag)==0:
                    one_bank_data = [str(item[1]),item[0].replace('&',' ')+' $\\rightarrow$',acquiror.replace('&',' ')+\
                        ' $\\rightarrow$',second_acquiror.replace('&',' ')]
                    table_data = table_data+[one_bank_data]
                else:
                    third_acquiror = MA_frag['acquiror'][0]
                    MA_frag = MA[MA['target']==third_acquiror].reset_index()
                    # Check if parent is acquired again
                    if len(MA_frag)==0:
                        one_bank_data = [str(item[1]),item[0].replace('&',' ')+' $\\rightarrow$',acquiror.replace('&',' ')+\
                            ' $\\rightarrow$',second_acquiror.replace('&',' ')+' $\\rightarrow$',third_acquiror.replace('&',' ')]
                        table_data = table_data+[one_bank_data]
                    else:
                        fourth_acquiror = MA_frag['acquiror'][0]
                        one_bank_data = [str(item[1]),item[0].replace('&',' ')+' $\\rightarrow$',acquiror.replace('&',' ')+\
                            ' $\\rightarrow$',second_acquiror.replace('&',' ')+' $\\rightarrow$',third_acquiror.replace('&',' ')+\
                            ' $\\rightarrow$',fourth_acquiror.replace('&',' ')]
                        table_data = table_data+[one_bank_data]
        elif len(MA_frag)==2:
            pass
    table_data = [row + [""]*(5-len(row)) for row in table_data]
    table_data = [['N Deals','Underwriter','First Acquiror','Second Acquiror','Third Acquiror','Fourth Acquiror']]+table_data
    
    latex_code = """
        """
    for row in table_data[:1]:
        latex_code += "\\vspace{2mm}"
        latex_code += " & ".join(row) + " \\\\\n"
    latex_code += "\\hline \\\\\n"
    for row in table_data[1:]:
        latex_code += "\\vspace{2mm}"
        latex_code += " & ".join(row) + " \\\\\n"
    
    # Save the LaTeX code to a .tex file
    with open("../Slides/tabs/fate_"+state+"_"+str(begin_year)+"_"+str(end_year)+"_"+str(begin_rank)+"_"+str(end_rank)+".tex", "w") as f:
        f.write(latex_code)


In [5]:
export_fate('MO',1980,1990,0,10)
export_fate('MO',1980,1990,10,20)
export_fate('MO',1990,2000,0,10)
export_fate('MO',1990,2000,10,20)
export_fate('MO',2000,2010,0,10)
export_fate('MO',2000,2010,10,20)
export_fate('MO',2010,2020,0,10)
export_fate('MO',2010,2020,10,20)
